In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import joblib
import os
import matplotlib.pyplot as plt
from keras.utils import plot_model

In [2]:
# settings
CURRENCY = "EUR"  # change this to GBP or JPY to run separately
CUTOFF_DATE = "2022-12-21"
PRED_END_DATE = "2025-02-01"
TIME_STEP = 10
EPOCHS = 50

In [3]:
# load data
df = pd.read_csv("../data/returns_2014_2025_filtered.csv", parse_dates=["Date"])
df = df.set_index("Date")
df = df[[CURRENCY]].copy()
df

,EUR
Date,
2014-01-02,0.006730
2014-01-03,0.005427
2014-01-04,0.000048
2014-01-05,-0.000408
2014-01-06,-0.002682
...,...
2025-01-30,-0.002783
2025-01-31,-0.003079
2025-02-03,-0.001544
